In [1]:
from keras.layers import Conv2D, MaxPooling2D, Input, Flatten, concatenate, Dense
from keras import Model
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import sys
import os

sys.path.append(os.path.join(os.getcwd(),"Preprocess"))
sys.path.append(os.path.join(os.getcwd(),"DataAcquisition"))

from helper import *
from transformers import *

init_config()

Using TensorFlow backend.
c:\users\kevin\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
def InceptionModule(x):
    tower_1 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)
    tower_1 = Conv2D(64, (3, tower_1.shape[2].value), padding='valid', activation='relu')(tower_1)

    tower_2 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)
    tower_2 = Conv2D(64, (5, tower_2.shape[2].value), padding='valid', activation='relu')(tower_2)

    tower_3 = MaxPooling2D((3, 1), strides=(1, 1), padding='valid')(x)
    tower_3 = Conv2D(64, (1, tower_3.shape[2].value), padding='valid', activation='relu')(tower_3)
    
    tower_4 = Conv2D(64, (1, x.shape[2].value), padding='valid', activation='relu')(x)

    print(tower_1.shape)
    print(tower_2.shape)
    print(tower_3.shape)
    print(tower_4.shape)
    
    return concatenate([tower_1, tower_2, tower_3, tower_4], axis=1)

In [31]:
def create_model(X, y):
    windowsize = X.shape[1]
    feature_len = X.shape[2]

    inputs = Input((windowsize, feature_len, 1, ))
    x = InceptionModule(inputs)
    x = Flatten()(inputs)
    x = Dense(200, activation="relu")(x)
    x = Dense(y.shape[1], activation="softmax")(x)
    outputs = x

    model = Model(inputs=inputs, outputs=outputs)

    model.compile(optimizer='rmsprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model


def evaluate_model(model, X_test, y_test):
    loss, metric = model.evaluate(X_test,y_test)
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    
    print("\nLoss: {} \nMetric: {}".format(loss, metric))
    print("\nConfusion Matrix:")
    for row in cm:
        print(row)

In [ ]:
def get_dummy_data():
    X_a = np.ones((1000, 100, 146, 1))
    y_a = np.array([[1,0]]*X_a.shape[0])

    X_b = np.zeros((1000, 100, 146, 1))
    y_b = np.array([[0,1]]*X_b.shape[0])

    X = np.vstack((X_a,X_b))
    y = np.concatenate((y_a,y_b))

    return train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)

In [ ]:
X_train, X_test, y_train, y_test = get_data(["wells", "chesterton"])

In [32]:
#model = create_model(X_train, y_train)
#model.fit(X_train, y_train, epochs=1, validation_split=.2, batch_size=32)
evaluate_model(model, X_test, y_test)

35536/35536 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 5s 148us/step

Loss: 0.4611561948358878 
Metric: 0.7870610085547051

Confusion Matrix:
[19811  2860]
[4707 8158]
